# **Introduction:**

This file serves to host an attempted Keras implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS).

**Date Created:** 22/01/2025

**Date Modified:** 23/01/2025

# **Import Packages:**

This section imports all the necessary packages for the ANFIS implementation.

In [4]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.layers import Layer

# **Function & Layer Definition:**

This section creates the necessary custom functions and layers for this ANFIS implementation within Keras. 

In [5]:
# need to first define the initial layer -> the membership function layer
class MembershipFunctionLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, params = None, **kwargs):   # by including **kwargs, we allow for additional arguments from keras, like name or dtype
        super(MembershipFunctionLayer, self).__init__(**kwargs)         # we are subclassing from the keras layer -> telling the constructor to make our layer like a keras layer
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

        if params is not None:
            # initialize custom parameters defined by the user:
            self.mf_params = self.add_weight(
                shape=(self.num_inputs, self.num_mfs, 3),
                initializer=tf.constant_initializer(params),
                trainable=True,
                name="mf_params",
            )
            print('Custom parameters have been set.')
        else:
            # initialize raw membership parameters:
            raw_params = self.add_weight(
                shape = (self.num_inputs, self.num_mfs, 3),
                initializer = "random_uniform",
                trainable = True,
                name = "Raw MF Params",
            )

            # sort the parameters such that a <= b <= c:
            sorted_params = tf.sort(raw_params, axis = 1)
            self.mf_params = self.add_weight(
                shape = (self.num_inputs, self.num_mfs, 3),
                initializer = tf.constant_initializer(sorted_params.numpy()),
                trainable = True, 
                name = 'MF Params'
            )
            print('Random parameters have been set.')

    # define the triangular membership function within this layer as this is where it is used:
    def triangular_membership(self, x, params):
        a, b, c = params

        if a > b or b > c:
            raise ValueError("Invalid parameters: Ensure a < b < c.")
    
        if a == b:  # rising ramp (plateau at b, c)
            return np.maximum(0, np.minimum(1, (c - x) / (c - b)))
        elif b == c:  # falling ramp (plateau at a, b)
            return np.maximum(0, np.minimum(1, (x - a) / (b - a)))
        
        # general triangular shape
        return np.maximum(0, np.minimum((x - a) / (b - a), (c - x) / (c - b)))
    
    # need to define the call -> this is what gets executed by the layer:
    def call(self, inputs):
        # initialize list to hold membership values:
        membership_values = []

        # loop through each input:
        for i in range(self.num_inputs):
            input_values = inputs[:, i]   # for a given column, everything in the row
            mf_values = []                # initialize list for the MF values of this input

            # for every membership function:
            for j in range(self.num_mfs):
                params = self.mf_params[i, j].numpy()  # extract params

                mf_values.append(np.array([self.triangular_membership(x, params) for x in input_values]))

            membership_values.append(np.stack(mf_values, axis=-1))  # stack MFs for this input

        # combine memberships for all inputs into a single tensor
        membership_values = tf.convert_to_tensor(np.stack(membership_values, axis=1), dtype=tf.float32)

        return membership_values


# **Test by Creating a Model:**

This section tests the designed layers by creating a model, adding the custom layers, and testing their operation. 

In [6]:
# define the following to be used in model generation:
num_inputs = 3
num_mfs = 3
params = np.array([
    [  # Parameters for input 1
        [0, 0, 6],
        [5/6, 5, 55/6],
        [4, 10, 10]
    ],
    [  # Parameters for input 2
        [0, 0 , 15],
        [25/12, 12.5, 275/12],
        [10, 25, 25]
    ],
    [  # Parameters for input 3
        [0, 0, 30],
        [25/6, 25, 275/6],
        [15, 50, 50]
    ]
])

# generate a model:
membership_layer = MembershipFunctionLayer(num_inputs = num_inputs, num_mfs = num_mfs, params = params)

inputs = tf.constant([[2, 9, 21], [8, 23, 48]], dtype=tf.float32)

membership_layer(inputs)

Custom parameters have been set.


<tf.Tensor: shape=(2, 3, 3), dtype=float32, numpy=
array([[[0.6666667 , 0.28000003, 0.        ],
        [0.4       , 0.66400003, 0.        ],
        [0.3       , 0.808     , 0.17142858]],

       [[0.        , 0.28000006, 0.6666667 ],
        [0.        , 0.        , 0.8666667 ],
        [0.        , 0.        , 0.94285715]]], dtype=float32)>